In [1]:
import numpy as np 

def load_dataset(filename):
    data_mat = []; label_mat = []
    with open(filename) as fr:
        for line in fr.readlines():
            line_arr = line.strip().split('\t')
            data_mat.append([float(line_arr[0]), float(line_arr[1])])
            label_mat.append(float(line_arr[2]))
    return data_mat, label_mat

def select_jrand(i, m):
    # 随机选择另一个优化的j,不等于i
    j = i 
    while j == i:
        j = np.random.randint(0, m)
    return j

def clip_alpha(aj, H, L):
    aj = max(aj, L)
    aj = min(aj, H)
    return aj

In [2]:
data_arr, label_arr = load_dataset('testSet.txt')
print(label_arr)

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [59]:
def smo_simple(data, classlabels, C, toler, max_iter):
    """
    C:常数
    toler:容错率
    """
    data_mat = np.mat(data)
    label_mat = np.mat(classlabels).transpose()
    b = 0; m,n = np.shape(data_mat)
    alphas = np.mat(np.zeros((m, 1)))
    num = 0
    while (num < max_iter):
        alpha_pair_changed = 0 
        for i in range(m):
            fXi = float(np.multiply(alphas, label_mat).T * \
                (data_mat * data_mat[i, :].T)) + b  # 计算出的类别
            Ei = fXi - float(label_mat[i])  # 误差Ei，根据此优化
            if ((label_mat[i] * Ei < -toler) and (alphas[i] < C)) or \
                ((label_mat[i] * Ei > toler) and (alphas[i] > 0) and \
                    (alphas[i] > 0)):   # 如果alpha可以改更改进入优化过程
                j = select_jrand(i, m)  # 随机选择第二个alpha
                fXj = float(np.multiply(alphas, label_mat).T * \
                    (data_mat * data_mat[j, :].T)) + b 
                Ej = fXj - float(label_mat[j])
                alpha_i_old = alphas[i].copy()
                alpha_j_old = alphas[j].copy() # 
                if (label_mat[i] != label_mat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H: 
                    # print("L==H")
                    continue
                eta = 2.0 * data_mat[i,:] * data_mat[i,:].T - \
                    data_mat[i,:]*data_mat[i,:].T - data_mat[j,:]*data_mat[j,:].T   # eta是alpha[j]的最优修改量
                if eta >=0: 
                    # print("eta>=0")
                    continue 
                alphas[j] -= label_mat[j]*(Ei - Ej)/eta
                alphas[j] = clip_alpha(alphas[j], H, L)
                if (abs(alphas[j] - alpha_j_old) < 0.00001):
                    # print("j not moving enough")
                    continue
                alphas[i] += label_mat[j] * label_mat[i] * (alpha_j_old - alphas[j])   # i的修改方向和j相反
                # 设置常数项
                b1 = b - Ei \
                    - label_mat[i] * (alphas[i] - alpha_i_old) * data_mat[i,:] * data_mat[i,:].T \
                    - label_mat[j] * (alphas[j] - alpha_j_old) * data_mat[i,:] * data_mat[j,:].T
                b2 = b - Ej \
                    - label_mat[i] * (alphas[i] - alpha_i_old) * data_mat[i,:] * data_mat[j,:].T \
                    - label_mat[j] * (alphas[j] - alpha_j_old) * data_mat[j,:] * data_mat[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1 
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2 
                else:
                    b = (b1 + b2) / 2.0
                alpha_pair_changed += 1
                # print("iter : %d i:%d, pairs changed %d" % \
                #     (num, i, alpha_pair_changed))
        if (alpha_pair_changed == 0): 
            num += 1
        else: 
            num = 0
        # print("iteration number: %d" % num)
    return b, alphas

In [60]:
b, alphas = smo_simple(data_arr, label_arr, 0.6, 0.001, 40)

In [61]:
b

matrix([[-3.95445957]])

In [62]:
alphas[alphas>0]

matrix([[5.55111512e-17, 9.50073353e-14, 8.04304540e-14, 6.95624114e-16,
         1.29588653e-01, 2.30116273e-13, 2.72351586e-15, 2.53015230e-01,
         5.52873719e-14, 2.18602046e-13, 5.55111512e-17, 3.82603883e-01,
         5.55111512e-17, 7.87044041e-15, 1.94757405e-14]])

In [63]:
for i in range(100):
    if alphas[i] > 0.0:
        print(data_arr[i], label_arr[i])

[2.326297, 0.265213] -1.0
[3.634009, 1.730537] -1.0
[3.125951, 0.293251] -1.0
[2.123252, -0.783563] -1.0
[4.658191, 3.507396] -1.0
[3.223038, -0.552392] -1.0
[2.301095, -0.533988] -1.0
[3.457096, -0.082216] -1.0
[3.023938, -0.057392] -1.0
[2.893743, -1.643468] -1.0
[1.870457, -1.04042] -1.0
[6.080573, 0.418886] 1.0
[2.529893, 0.662657] -1.0
[1.966279, -1.840439] -1.0
[2.912122, -0.202359] -1.0


In [64]:
# 利用完整的platt SMO 优化算法

In [65]:
class DataStruct:
    def __init__(self, data_mat, classlabels, C, toler):
        self.X = data_mat
        self.label_mat = classlabels
        self.C = C 
        self.tol = toler 
        self.m = np.shape(data_mat)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.e_cache = np.mat(np.zeros((self.m, 2)))


def calc_ek(ds, k):
    fXk = np.multiply(ds.alphas, ds.label_mat).T *\
        (ds.X * ds.X[k,:].T) + ds.b
    Ek = float(fXk[0][0]) - float(ds.label_mat[k])
    return Ek

def select_j(i, ds, Ei):
    max_k = -1
    max_deltaE = 0
    Ej = 0  # 内循环中的启发式方法
    ds.e_cache[i] = [1, Ei]
    valid_Ecache_lst = np.nonzero(np.array(ds.e_cache[:, 0]))[0]
    if (len(valid_Ecache_lst) > 1):
        # >1说明至之前计算过别的E，选择最大的进行优化
        for k in valid_Ecache_lst:
            if k == i:
                continue
            Ek = calc_ek(ds, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > max_deltaE):
                max_k = k
                max_deltaE = deltaE 
        return max_k, Ej 
    else:
        # 第一次循环就直接随机选择一个j
        j = select_jrand(i, ds.m)
        Ej = calc_ek(ds, j)
    return j, Ej 


def updateEk(ds, k):
    Ek = calc_ek(ds, k)
    ds.e_cache[k] = [1, Ek]

In [66]:
def innerL(i, ds):
    Ei = calc_ek(ds, i)
    if ((ds.label_mat[i] * Ei < -ds.tol) and (ds.alphas[i] < ds.C)) or \
        ((ds.label_mat[i] * Ei > ds.tol) and (ds.alphas[i] > 0)):
        j, Ej = select_j(i, ds, Ei) # 第二个alpha选择中的启发式方法
        alpha_i_old = ds.alphas[i].copy()
        alpha_j_old = ds.alphas[j].copy()
        if (ds.label_mat[i] != ds.label_mat[j]):
            L = max(0, ds.label_mat[j] - ds.label_mat[i])
            H = min(ds.C, ds.C + ds.label_mat[j] - ds.label_mat[i])
        else:
            L = max(0, ds.label_mat[j] + ds.label_mat[i] - ds.C)
            H = min(ds.C, ds.label_mat[j] + ds.label_mat[i])
        if L == H:
            # print("L==H")
            return 0 
        eta = 2.0 * ds.X[i,:] * ds.X[i,:].T - \
                    ds.X[i,:]*ds.X[i,:].T - ds.X[j,:]*ds.X[j,:].T
        if eta >=0: 
            # print("eta>=0")
            return 0 
        ds.alphas[j] -= ds.label_mat[j]*(Ei - Ej)/eta
        ds.alphas[j] = clip_alpha(ds.alphas[j], H, L)
        updateEk(ds, j)
        if (abs(alphas[j] - alpha_j_old) < 0.00001):
            # print("j not moving enough")
            return 0
        ds.alphas[i] += ds.label_mat[j] * ds.label_mat[i] * (alpha_j_old - ds.alphas[j])
        updateEk(ds, i) # 更新误差缓存
        b1 = ds.b - Ei \
            - ds.label_mat[i] * (ds.alphas[i] - alpha_i_old) * ds.X[i,:] * ds.X[i,:].T \
            - ds.label_mat[j] * (ds.alphas[j] - alpha_j_old) * ds.X[i,:] * ds.X[j,:].T
        b2 = ds.b - Ej \
            - ds.label_mat[i] * (ds.alphas[i] - alpha_i_old) * ds.X[i,:] * ds.X[j,:].T \
            - ds.label_mat[j] * (ds.alphas[j] - alpha_j_old) * ds.X[j,:] * ds.X[j,:].T
        if (0 < ds.alphas[i]) and (ds.C > ds.alphas[i]):
            ds.b = b1 
        elif (0 < ds.alphas[j]) and (ds.C > ds.alphas[j]):
            ds.b = b2 
        else:
            ds.b = (b1 + b2) / 2.0
        return 1 
    else:
        return 0

In [67]:
def smo_p(data_mat, classlabels, C, toler, max_iter, kTup=('lin', 0)):
    """完整SMO算法中的外循环代码"""
    ds = DataStruct(np.mat(data_mat), np.mat(classlabels).transpose(), C, toler, kTup)
    num = 0
    entire_set = True 
    alpha_pairs_changed = 0
    while (num < max_iter) and ((alpha_pairs_changed > 0) or (entire_set)):
        alpha_pairs_changed = 0
        if entire_set:  # 遍历所有值
            for i in range(ds.m):
                alpha_pairs_changed += innerL(i, ds)
            # print("Fullset, iter: %d i: %d, pairs changed %d" %\
            #     (num, i, alpha_pairs_changed))
            num += 1
        else:       # 遍历非边界值
            non_bounds = np.nonzero((np.array(ds.alphas) > 0) * (np.array(ds.alphas) < C))[0]
            for i in non_bounds:
                alpha_pairs_changed += innerL(i, ds)
                # print("non-bound, iter: %d i: %d, pairs changed %d" %\
                #     (num, i, alpha_pairs_changed))
                num += 1
        if entire_set:
            entire_set = False 
        elif alpha_pairs_changed == 0:
            entire_set = True 
        # print("iteration number: %d" % num)
    return ds.b, ds.alphas

In [68]:
data_arr, label_arr = load_dataset('testSet.txt')
b, alphas = smo_p(data_arr, label_arr, 0.6, 0.001, 40)

TypeError: __init__() takes 5 positional arguments but 6 were given

In [69]:
b

matrix([[-3.95445957]])

In [70]:
alphas[alphas>0.0]

matrix([[5.55111512e-17, 9.50073353e-14, 8.04304540e-14, 6.95624114e-16,
         1.29588653e-01, 2.30116273e-13, 2.72351586e-15, 2.53015230e-01,
         5.52873719e-14, 2.18602046e-13, 5.55111512e-17, 3.82603883e-01,
         5.55111512e-17, 7.87044041e-15, 1.94757405e-14]])

In [71]:
for i in range(100):
    if alphas[i] > 0.0:
        print(data_arr[i], label_arr[i])

[2.326297, 0.265213] -1.0
[3.634009, 1.730537] -1.0
[3.125951, 0.293251] -1.0
[2.123252, -0.783563] -1.0
[4.658191, 3.507396] -1.0
[3.223038, -0.552392] -1.0
[2.301095, -0.533988] -1.0
[3.457096, -0.082216] -1.0
[3.023938, -0.057392] -1.0
[2.893743, -1.643468] -1.0
[1.870457, -1.04042] -1.0
[6.080573, 0.418886] 1.0
[2.529893, 0.662657] -1.0
[1.966279, -1.840439] -1.0
[2.912122, -0.202359] -1.0


In [72]:
def calc_ws(alphas, data_arr, classlabels):
    X = np.mat(data_arr)
    label_mat = np.mat(classlabels).transpose()
    m,n = np.shape(X)
    w = np.zeros((n,1))
    for i in range(m):
        w += np.multiply(alphas[i] * label_mat[i], X[i,:].T)
    return w

In [73]:
ws = calc_ws(alphas, data_arr, label_arr)

In [74]:
ws

array([[ 0.8481042 ],
       [-0.27344941]])

In [75]:
label_arr[0]

-1.0

In [76]:
data_arr[1] * np.mat(ws) + b

matrix([[-2.09317163]])

In [77]:
label_arr[1]

-1.0

# 核函数

In [3]:
import numpy as np 

def load_dataset(filename):
    data_mat = []; label_mat = []
    with open(filename) as fr:
        for line in fr.readlines():
            line_arr = line.strip().split('\t')
            data_mat.append([float(line_arr[0]), float(line_arr[1])])
            label_mat.append(float(line_arr[2]))
    return data_mat, label_mat

def select_jrand(i, m):
    # 随机选择另一个优化的j,不等于i
    j = i 
    while j == i:
        j = np.random.randint(0, m)
    return j

def clip_alpha(aj, H, L):
    aj = max(aj, L)
    aj = min(aj, H)
    return aj

def calc_ek(ds, k):
    fXk = np.multiply(ds.alphas, ds.label_mat).T *\
        (ds.X * ds.X[k,:].T) + ds.b
    Ek = float(fXk[0][0]) - float(ds.label_mat[k])
    return Ek

def select_j(i, ds, Ei):
    max_k = -1
    max_deltaE = 0
    Ej = 0  # 内循环中的启发式方法
    ds.e_cache[i] = [1, Ei]
    valid_Ecache_lst = np.nonzero(np.array(ds.e_cache[:, 0]))[0]
    if (len(valid_Ecache_lst) > 1):
        # >1说明至之前计算过别的E，选择最大的进行优化
        for k in valid_Ecache_lst:
            if k == i:
                continue
            Ek = calc_ek(ds, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > max_deltaE):
                max_k = k
                max_deltaE = deltaE 
        return max_k, Ej 
    else:
        # 第一次循环就直接随机选择一个j
        j = select_jrand(i, ds.m)
        Ej = calc_ek(ds, j)
    return j, Ej 


def updateEk(ds, k):
    Ek = calc_ek(ds, k)
    ds.e_cache[k] = [1, Ek]

In [4]:
def kernel_trans(X, A, ktup):
    m,n = np.shape(X)
    K = np.mat(np.zeros((m, 1)))
    if ktup[0] == 'lin':
        K = X * A.T 
    elif ktup[0] == 'rbf':
        for j in range(m):
            delta_row = X[j, :] - A 
            K[j] = delta_row * delta_row.T 
        K = np.exp(K / (-1*ktup[1]**2)) # 元素间的除法
    else:
        raise NameError('Houston We Have a Problem --\
            That Kernel is not recognized')
    return K

In [5]:
class DataStruct:
    def __init__(self, data_mat, classlabels, C, toler, ktup):
        self.X = data_mat
        self.label_mat = classlabels
        self.C = C 
        self.tol = toler 
        self.m = np.shape(data_mat)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.e_cache = np.mat(np.zeros((self.m, 2)))
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:,i] = kernel_trans(self.X, self.X[i,:], ktup)

In [38]:
def innerL(i, ds):
    Ei = calc_ek(ds, i)
    if ((ds.label_mat[i] * Ei < -ds.tol) and (ds.alphas[i] < ds.C)) or \
        ((ds.label_mat[i] * Ei > ds.tol) and (ds.alphas[i] > 0)):
        j, Ej = select_j(i, ds, Ei) # 第二个alpha选择中的启发式方法
        alpha_i_old = ds.alphas[i].copy()
        alpha_j_old = ds.alphas[j].copy()
        if (ds.label_mat[i] != ds.label_mat[j]):
            L = max(0, ds.label_mat[j] - ds.label_mat[i])
            H = min(ds.C, ds.C + ds.label_mat[j] - ds.label_mat[i])
        else:
            L = max(0, ds.label_mat[j] + ds.label_mat[i] - ds.C)
            H = min(ds.C, ds.label_mat[j] + ds.label_mat[i])
        if L == H:
            # print("L==H")
            return 0 
        eta = 2.0 * ds.K[i,j] - ds.K[i,i] - ds.K[j,j]
        if eta >=0: 
            # print("eta>=0")
            return 0 
        ds.alphas[j] -= ds.label_mat[j]*(Ei - Ej)/eta
        ds.alphas[j] = clip_alpha(ds.alphas[j], H, L)
        updateEk(ds, j)
        if (abs(ds.alphas[j] - alpha_j_old) < 0.00001):
            # print("j not moving enough")
            return 0
        ds.alphas[i] += ds.label_mat[j] * ds.label_mat[i] * (alpha_j_old - ds.alphas[j])
        updateEk(ds, i) # 更新误差缓存
        b1 = ds.b - Ei \
            - ds.label_mat[i] * (ds.alphas[i] - alpha_i_old) * ds.K[i,i] \
            - ds.label_mat[j] * (ds.alphas[j] - alpha_j_old) * ds.K[i,j]
        b2 = ds.b - Ej \
            - ds.label_mat[i] * (ds.alphas[i] - alpha_i_old) * ds.K[i,j] \
            - ds.label_mat[j] * (ds.alphas[j] - alpha_j_old) * ds.K[j,j]
        if (0 < ds.alphas[i]) and (ds.C > ds.alphas[i]):
            ds.b = b1 
        elif (0 < ds.alphas[j]) and (ds.C > ds.alphas[j]):
            ds.b = b2 
        else:
            ds.b = (b1 + b2) / 2.0
        return 1 
    else:
        return 0


def calc_ek(ds, k):
    fXk = np.multiply(ds.alphas, ds.label_mat).T *\
        ds.K[:,k] + ds.b
    Ek = float(fXk[0][0]) - float(ds.label_mat[k])
    return Ek

def smo_p(data_mat, classlabels, C, toler, max_iter, kTup=('lin', 0)):
    ds = DataStruct(np.mat(data_mat), np.mat(classlabels).transpose(), C, toler, kTup)
    num = 0
    entire_set = True 
    alpha_pairs_changed = 0
    while (num < max_iter) and ((alpha_pairs_changed > 0) or (entire_set)):
        alpha_pairs_changed = 0
        if entire_set:  # 遍历所有值
            for i in range(ds.m):
                alpha_pairs_changed += innerL(i, ds)
            num += 1
        else:       # 遍历非边界值
            non_bounds = np.nonzero((ds.alphas.A > 0) * (ds.alphas.A < C))[0]
            for i in non_bounds:
                alpha_pairs_changed += innerL(i, ds)
            num += 1
        if entire_set:
            entire_set = False 
        elif alpha_pairs_changed == 0:
            entire_set = True 
    return ds.b, ds.alphas

In [44]:
# 利用核函数进行分类的径向基测试函数
def test_rbf(k1 = 1.3):
    data_arr, label_arr = load_dataset('testSetRBF.txt')
    b, alphas = smo_p(data_arr, label_arr, 200, 0.0001, 100, ('rbf', k1))
    data_mat = np.mat(data_arr)
    label_mat = np.mat(label_arr).transpose()
    svInd = np.nonzero(np.array(alphas) > 0)[0]   # 构建支持向量矩阵
    sVs = data_mat[svInd]
    labelSV = label_mat[svInd]
    print("There are %d Support Vectors" % sVs.shape[0])
    m, n = data_mat.shape
    error_count = 0
    for i in range(m):
        kernel_eval = kernel_trans(sVs, data_mat[i,:], ('rbf', k1))
        predict = kernel_eval.T * np.multiply(labelSV, alphas[svInd]) + b 
        if np.sign(predict[0][0]) != np.sign(label_arr[i]):
            error_count += 1
    print("The training error rate is %f" % (float(error_count) / m))

    data_arr, label_arr = load_dataset('testSetRBF2.txt')
    error_count = 0
    data_mat = np.mat(data_arr)
    label_mat = np.mat(label_arr)
    m, n = data_mat.shape
    for i in range(m):
        kernel_eval = kernel_trans(sVs, data_mat[i,:], ('rbf', k1))
        predict = kernel_eval.T * np.multiply(labelSV, alphas[svInd]) + b 
        if np.sign(predict[0][0]) != np.sign(label_arr[i]):
            error_count += 1
    print("The test error rate is %f" %(float(error_count) / m))


In [45]:
test_rbf(k1=1.3)

There are 27 Support Vectors
The training error rate is 0.440000
The test error rate is 0.370000


In [49]:
test_rbf(k1=1)

There are 17 Support Vectors
The training error rate is 0.040000
The test error rate is 0.060000


# 手写字符识别问题

In [54]:
def img2vector(filename):
    """将手写的数字图片转化为向量"""
    return_vec = np.zeros((1, 1024))
    with open(filename) as fr:  # python这样打开文件不用再close()
        for i in range(32):
            line_str = fr.readline()
            for j in range(32):
                return_vec[0, 32*i+j] = int(line_srt[j])
    
    return return_vec

def load_images(dirname):
    from os import listdir 
    hw_labels = []
    training_file_lst = listdir(dirname)
    m = len(training_file_lst)
    training_mat = np.zeros((m, 1024))
    for i in range(m):
        filename = training_file_lst[i]
        filestr = filename.split('.')[0]
        class_num = int(filestr.split('_')[0])
        if class_num == 9:
            hw_labels.append(-1)
        else:
            hw_labels.append(1)
        training_mat[i,:] = img2vector('%s/%s' %(dirname, filename))
    return training_mat, hw_labels

In [55]:
def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = load_images('trainingDigits')
    b,alphas = smo_p(dataArr, labelArr, 200, 0.0001, 10, kTup)
    datMat=np.mat(dataArr); labelMat = np.mat(labelArr).transpose()
    svInd=np.nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd]
    print ("there are %d Support Vectors" % shape(sVs)[0])
    m,n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernel_trans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))

    dataArr,labelArr = load_images('testDigits')
    errorCount = 0
    datMat=np.mat(dataArr); labelMat = np.mat(labelArr).transpose()
    m,n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernel_trans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if np.sign(predict)!=np.sign(labelArr[i]): errorCount += 1    
    print ("the test error rate is: %f" % (float(errorCount)/m) )

In [56]:
testDigits(kTup=('rbf', 10))

FileNotFoundError: [Errno 2] No such file or directory: 'trainingDigits/5_135.txt'